In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
os.listdir('/kaggle/input')

['efficientnetpytorchpackage', 'aptos-models', 'aptos2019-blindness-detection']

In [2]:
import sys
package_path = '../input/efficientnetpytorchpackage/EfficientNet-PyTorch-master'
sys.path.append(package_path)
from efficientnet_pytorch import EfficientNet

In [3]:
from fastai.vision import *
import cv2
import fastai


print(torch.__version__)
print(fastai.__version__)

1.2.0
1.0.57


# Function Definitions

In [4]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [5]:


def get_prob(arr):
    #print(arr.shape)
    
    probs=torch.zeros(arr.size(0)+1,dtype=arr.dtype)
    probs[4]=arr[3]
    probs[3]=(arr[2]-arr[3]).relu()
    probs[2]=(arr[1]-arr[2]).relu()
    probs[1]=(arr[0]-arr[1]).relu()
    probs[0]=1-arr[0]

    return probs

def get_pred(probs):
    #print(arr.shape)
    return probs.argmax(0)

In [6]:
def clahe(img):
    
    clahe = cv2.createCLAHE(clipLimit=2 , tileGridSize=(8,8))
    img_new_1 = clahe.apply(img[:,:,0])
    img_new_2 = clahe.apply(img[:,:,1])
    img_new_3 = clahe.apply(img[:,:,2])
    img_merge = cv2.merge([img_new_1,img_new_2,img_new_3])
    
    return img_merge

def clahel(img):
    
    image = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    clahe = cv2.createCLAHE(clipLimit=2 , tileGridSize=(8,8))
    img_new_1 = clahe.apply(image[:,:,0])
    img_merge = cv2.merge([img_new_1,image[:,:,1],image[:,:,2]])
    
    return cv2.cvtColor(img_merge,cv2.COLOR_LAB2RGB)


In [7]:
def img_load(p,IMG_SIZE=512):
    image = cv2.imread(p)
    image = crop_image_from_gray(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width = image.shape[:2]
    m=max(height,width)
    
    f=IMG_SIZE/m
    image = cv2.resize(image, None,fx=f, fy=f)
    
    return image


def img_proc(image,blur_rad=10,use_clahel=False,use_clahe=False):
    
    if(use_clahel==True):
        image = clahel(image)
    if(use_clahe==True):
        image = clahe(image)
    
    if (blur_rad>0):
        image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0), blur_rad) ,-4 ,128)
    return Image(pil2tensor(image, np.float32).div_(255))

# Load Models

In [8]:
model_path='/kaggle/input/aptos-models/'
#No Processing
learn1=load_learner(model_path,'finalresnet34-all_train_images_512.pkl')
learn2=load_learner(model_path,'finalefficientnet-b6-all_train_images_512.pkl')

#Blur=10
learn0=load_learner(model_path,'final-rn34-nonzero.pkl')
#learn3=load_learner(model_path,'finalresnet34-all_train_images_512_blur.pkl')
#learn4=load_learner(model_path,'finalefficientnet-b6-all_train_images_512_blur.pkl')

#Clahe
learn5=load_learner(model_path,'finalresnet34-all_train_images_512_clahe.pkl')
learn6=load_learner(model_path,'finalefficientnet-b6-all_train_images_512_clahe.pkl')

#Clahel
learn7=load_learner(model_path,'finalresnet34-all_train_images_512_clahel.pkl')
learn8=load_learner(model_path,'finalefficientnet-b6-all_train_images_512_clahel.pkl')

#Blur=50
#learn9=load_learner(model_path,'finalresnet34-all_train_images_512_blur_50.pkl')
#learn10=load_learner(model_path,'finalefficientnet-b6-all_train_images_512_blur_50.pkl')

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'efficientnet_pytorch.model.EfficientNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


# Predict Test Set

In [9]:
sample_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
sample_df.head()

,id_code,diagnosis
0,0005cfc8afb6,0
1,003f0afdcd15,0
2,006efc72b638,0
3,00836aaacf06,0
4,009245722fa4,0


In [10]:
IMG_SIZE=512


test_path='/kaggle/input/aptos2019-blindness-detection/test_images/'
for it in progress_bar(sample_df.index):
    fns=str(sample_df.id_code[it])+'.png'

    p=test_path + fns

    image = cv2.imread(p)
    image = crop_image_from_gray(image)
    height, width = image.shape[:2]    
    m=max(height,width)            
    f=IMG_SIZE/m
    image_no_proc = cv2.resize(image, None,fx=f, fy=f)
    image_no_proc = cv2.cvtColor(image_no_proc, cv2.COLOR_BGR2RGB)
    image_blur_10=cv2.addWeighted ( image_no_proc,4, cv2.GaussianBlur( image_no_proc , (0,0), 10) ,-4 ,128)
    image_clahe = clahe(image_no_proc)
    image_clahel = clahel(image_no_proc)
    #image_blur_50 = cv2.addWeighted ( image_no_proc,4, cv2.GaussianBlur( image_no_proc , (0,0), 50) ,-4 ,128)

    
    probs=0
    
    #no processing
    imagef=Image(pil2tensor(image_no_proc, np.float32).div_(255))  
    
    pred_class,pred_idx,outputs = learn1.predict(imagef)
    probs=probs+get_prob(outputs)
    
    pred_class,pred_idx,outputs = learn2.predict(imagef)
    probs=probs+get_prob(outputs)
    
    #blur=10
    imagef=Image(pil2tensor(image_blur_10, np.float32).div_(255))
    
    pred_class,pred_idx,outputs = learn0.predict(imagef)
    probs=probs+get_prob(outputs)
    
    #pred_class,pred_idx,outputs = learn3.predict(imagef)
    #probs=probs+get_prob(outputs)
    
    #pred_class,pred_idx,outputs = learn4.predict(imagef)
    #probs=probs+get_prob(outputs)
    
    #clahe
    imagef=Image(pil2tensor(image_clahe, np.float32).div_(255))
    pred_class,pred_idx,outputs = learn5.predict(imagef)
    probs=probs+get_prob(outputs)
    
    pred_class,pred_idx,outputs = learn6.predict(imagef)
    probs=probs+get_prob(outputs)
    
    #clahel
    imagef=Image(pil2tensor(image_clahel, np.float32).div_(255))
    pred_class,pred_idx,outputs = learn7.predict(imagef)
    probs=probs+get_prob(outputs)
    
    pred_class,pred_idx,outputs = learn8.predict(imagef)
    probs=probs+get_prob(outputs)
    
    #blur=50
    
    #imagef=Image(pil2tensor(image_blur_50, np.float32).div_(255))
    #pred_class,pred_idx,outputs = learn9.predict(imagef)
    #probs=probs+get_prob(outputs)
    
    #pred_class,pred_idx,outputs = learn10.predict(imagef)
    #probs=probs+get_prob(outputs)
    
    preds2=get_pred(probs)
    #print(pred_class, pred_idx,outputs[pred_idx])
    #sample_df.diagnosis[it]=preds2.numpy()
    sample_df.loc[it,'diagnosis']=preds2.numpy()
sample_df[0:20]

,id_code,diagnosis
0,0005cfc8afb6,2
1,003f0afdcd15,2
2,006efc72b638,2
3,00836aaacf06,2
4,009245722fa4,2
5,009c019a7309,2
6,010d915e229a,3
7,0111b949947e,1
8,01499815e469,2
9,0167076e7089,0


In [11]:
sample_df.to_csv('submission.csv',index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 